Imports

In [57]:
from models import ff_eucl, ff_hyp
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import helper
torch.cuda.is_available()

False

CUDA check

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {}'.format(device))

Using cpu


Data Transformation

In [59]:
transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Normalize((0.1307,), (0.3081,)), 
                              ])

Training and Test data from MNIST data set

In [60]:
train_set = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
test_set = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

size = len(train_set)
print(size)

train_data, val_data = torch.utils.data.random_split(train_set, [int(size-size*0.2), int(size*0.2)])

# trainloader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True)
# valloader = torch.utils.data.DataLoader(val_data, batch_size=512, shuffle= True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=512, shuffle= True)

60000


Initializing the model

In [61]:
# model = ff_eucl.EuclFF(784, 512, 256, 10, nn.ReLU())
# ball = geoopt.PoincareBall()
# images, labels = next(iter(trainloader))
# images = images.view(images.shape[0], -1)
# grid = torchvision.utils.make_grid(images)
# tb = SummaryWriter()
# tb.add_image("images", grid)
# tb.add_graph(model, images)
# model.to(device)

Hyperparameters

In [62]:

# learning_rate = 4e-1 #learning rate for ReLU activation function
# #learning_rate = 2e-1 #current learning rate for model without activation functions
# momentum = 0.9
# weight_decay = 5e-4

In [63]:
from itertools import product
parameters = dict(
    lr = [0.1, 0.01, 0.001],
    batch_size = [64,128,256,512],
    shuffle = [True, False]
)

param_values = [v for v in parameters.values()]
print(param_values)

for lr,batch_size, shuffle in product(*param_values):
    print(lr, batch_size, shuffle)

[[0.01, 0.001], [32, 64, 128], [True, False]]
0.01 32 True
0.01 32 False
0.01 64 True
0.01 64 False
0.01 128 True
0.01 128 False
0.001 32 True
0.001 32 False
0.001 64 True
0.001 64 False
0.001 128 True
0.001 128 False


Loss Function

In [64]:
#criterion = nn.CrossEntropyLoss()
# #criterion = nn.NLLLoss()
# images, labels = next(iter(trainloader))
# images, labels = images.to(device), labels.to(device)
# images = 0.0357*images.view(images.shape[0], -1)
# print(images)

# out = model(images) #output
# print(out)
# loss = criterion(out, labels) #calculate the loss

Optimizer

In [65]:
#optimizer = geoopt.optim.RiemannianSGD(model.parameters(), lr=learning_rate, momentum=momentum)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

In [66]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [67]:
for run_id, (lr,batch_size, shuffle) in enumerate(product(*param_values)):
    print("run id:", run_id + 1)
    model = ff_eucl.EuclFF(784, 512, 256, 10, nn.ReLU())
    trainloader = torch.utils.data.DataLoader(train_data,batch_size = batch_size, shuffle = shuffle)
    valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle= shuffle)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    comment = f' batch_size = {batch_size} lr = {lr} shuffle = {shuffle}'
    tb = SummaryWriter(comment=comment)
# time0 = time()
    epochs = 10
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        total_correct = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            # Flatten MNIST images into a 784 long vector
            images = images.view(images.shape[0], -1)
            #images = ball.projx(images.view(images.shape[0], -1))
            # Training pass
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)  
            train_loss += loss.item()
            total_correct += get_num_correct(output, labels)
            #backpropagation
            loss.backward()      
            #Weight optimization
            optimizer.step()  

        val_loss = 0
        val_correct = 0
        model.eval()
        for  images, labels in valloader:
            images, labels = images.to(device), labels.to(device)
            # Flatten MNIST images into a 784 long vector
            images = images.view(images.shape[0], -1)
            #images = ball.projx(images.view(images.shape[0], -1))
            output = model(images)
            loss = criterion(output, labels)  
            val_loss += loss.item()
            val_correct += get_num_correct(output, labels)
            
        tb.add_scalar("Training Loss", train_loss, epoch)
        tb.add_scalar("Validation Loss", val_loss, epoch)
        tb.add_scalar("Training Accuracy", total_correct/len(train_data), epoch)
        tb.add_scalar("Validation Accuracy", val_correct/len(val_data), epoch)

        print("epoch:", epoch, "training loss:",train_loss, "validation loss:", val_loss,
        "training accuracy:", total_correct/len(train_data), "validation accuracy:", val_correct/len(val_data))
    
    tb.add_hparams(
            {"lr": lr, "bsize": batch_size, "shuffle":shuffle},
            {
                "training accuracy": total_correct/ len(train_data),
                "validation accuracy": val_correct/ len(val_data),
                "training loss": train_loss,
                "validation loss": val_loss,
            },
        )
tb.close()
#     else:
#         print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
# print("\nTraining Time (in minutes) =",(time()-time0)/60)

run id: 1
epoch: 0 training loss: 1789.9757485687733 validation loss: 179.70880576968193 training accuracy: 0.7256458333333333 validation accuracy: 0.8701666666666666
epoch: 1 training loss: 584.6666019707918 validation loss: 132.52207375317812 training accuracy: 0.8904166666666666 validation accuracy: 0.899
epoch: 2 training loss: 477.6259970664978 validation loss: 118.41342249512672 training accuracy: 0.9091666666666667 validation accuracy: 0.91075
epoch: 3 training loss: 421.40932697430253 validation loss: 105.22179204970598 training accuracy: 0.9198125 validation accuracy: 0.9196666666666666
epoch: 4 training loss: 376.2325002383441 validation loss: 95.60663674771786 training accuracy: 0.9276041666666667 validation accuracy: 0.92725
epoch: 5 training loss: 337.9848978854716 validation loss: 89.10760646872222 training accuracy: 0.9350416666666667 validation accuracy: 0.9318333333333333
epoch: 6 training loss: 304.77749324962497 validation loss: 80.79697088524699 training accuracy: 0

In [68]:
# epochs = 10
# for epoch in range(epochs):
#     model.eval()
#     val_loss = 0
#     correct = 0
    
#     with torch.no_grad():
#         for images, labels in valloader:
#             images, labels = images.to(device), labels.to(device)
#             # Flatten MNIST images into a 784 long vector
#             images = images.view(images.shape[0], -1)
#             #images = ball.projx(images.view(images.shape[0], -1))
#             output = model(images)
#             loss = criterion(output, labels)  
#             val_loss += loss.item()
#             correct += get_num_correct(output, labels)

#         tb.add_scalar("Validation Loss", val_loss, epoch)
#         tb.add_scalar("Validation Accuracy", correct/len(val_data), epoch)
#         print("epoch:", epoch, "total correct:", correct, "validation loss:",val_loss)
#         print("Validation Accuracy", correct/len(val_data))
# tb.close()

In [69]:
# images, labels = next(iter(testloader))
# #images, labels = images.to(device), labels.to(device)

# img = images[0].view(1, 784)
# #img = ball.projx(images[0].view(1, 784))
# #img_gpu = img.to(device)
# with torch.no_grad():
#     out = model(img)

# ps = out.cpu()
# print(ps)
# probab = list(ps.numpy()[0])
# print(probab)
# print("Predicted Digit =", probab.index(max(probab)))
# helper.view_classify(img.view(1, 28, 28), ps)

#Model Prediction and Model Accuracy

In [70]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  images, labels = images.to(device), labels.to(device)
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    #img = ball.projx(images[i].view(1, 784))
    with torch.no_grad():
        out = model(img)

    ps = out.cpu()
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu().numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1
    
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.6335


Current status of experiments:
1. Using just Hyperboic Linear modules, and with the appropriate self-tuned hyperparameters, and a batch size of 512, the average accuracy was around 90 percent
2. With the use of activation functions(ReLu, ReLu, then LogSoftMax at the output layer), (by applying the functions in the tangent space, then mapping it back to the hyperbolic space), we see an increase in the model accuracy to about 97-98 percent.
3. To account for the correct class probabilities , linear layer was used as the output layer instead, together with the crossentropy loss function.